# **Phase 1 - Analysing GoEmotions and VAD Mapping**

## GoEmotions DataSet

*   [GoEmotions @ Google's github](https://github.com/google-research/google-research/tree/master/goemotions/data)




In [48]:
!pip3 install datasets transformers -q

In [49]:
# Load split GoEmotions to pandas DFs
from datasets import load_dataset
go_emotions = load_dataset("go_emotions", "simplified")
data = go_emotions.data
train, dev, test = data["train"].to_pandas(), data["validation"].to_pandas(), data["test"].to_pandas()

Reusing dataset go_emotions (/root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d)


  0%|          | 0/3 [00:00<?, ?it/s]

In [50]:
# Adding verbal emotions column

# TODO: fetch the following list from data/emotions.txt
labels_list = [
  'admiration',
  'amusement',
  'anger',
  'annoyance',
  'approval',
  'caring',
  'confusion',
  'curiosity',
  'desire',
  'disappointment',
  'disapproval',
  'disgust',
  'embarrassment',
  'excitement',
  'fear',
  'gratitude',
  'grief',
  'joya',
  'love',
  'nervousness',
  'optimism',
  'pride',
  'realization',
  'relief',
  'remorse',
  'sadness',
  'surprise',
  'neutral'
]
idx2emotion = {i : e for i, e in enumerate(labels_list)}
emotion2idx = None # TODO reverse the dict

train["emotions"] = train["labels"].apply(lambda labels: [idx2emotion[i] for i in labels])
dev["emotions"] = dev["labels"].apply(lambda labels: [idx2emotion[i] for i in labels])
test["emotions"] = test["labels"].apply(lambda labels: [idx2emotion[i] for i in labels])


## Delete multi-labels from datasets

We shall temporarily ignore multi-labaled exmaple just for simplicity. Later we will choose a way to treat them

There might be another way of doing it, as @Shir said.

In [51]:
# Add label counts column
train['labels_count'] = train['labels'].apply(lambda labels: len(labels))
dev['labels_count'] = dev['labels'].apply(lambda labels: len(labels))
test['labels_count'] = test['labels'].apply(lambda labels: len(labels))

# Print percenrage of multi-labels (candidates to delete)
print("--- What is the part that will be delete? --")
print("train : ", sum(train['labels_count'] > 1) / len(train['labels_count']))
print("dev : ", sum(dev['labels_count'] > 1) / len(dev['labels_count']))
print("test : ", sum(test['labels_count'] > 1) / len(test['labels_count']))

--- What is the part that will be delete? --
train :  0.16360285648468095
dev :  0.1618134906008109
test :  0.15422885572139303


In [52]:
train_one_label = train[train['labels_count'] == 1]
dev_one_label = dev[dev['labels_count'] == 1]
test_one_label = test[test['labels_count'] == 1]

## Map to VAD
We'd like to map each example to VAD, using its label (= emotion)

1. **First option** - [NRC-VAD : 2018](http://saifmohammad.com/WebPages/nrc-vad.html)
Crowd sourced lexicon, mapping 20K words to VAD.

In [55]:
# TODO fetch mapping from `data/mapping-emotions-to-vad/nrc-vad/NRC-VAD-Lexicon.txt`

2. **Second option** - [ANEW : 2018](https://pdodds.w3.uvm.edu/teaching/courses/2009-08UVM-300/docs/others/everything/bradley1999a.pdf)


// TODO add description

In [54]:
# TODO map (possibly manually) from the PDF to vad